# 75 persons
## electrodes : 8, 16, 32, 64

In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

2023-10-01 13:16:43.105001: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-01 13:16:43.152581: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-01 13:16:43.152629: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-01 13:16:43.152654: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-01 13:16:43.160300: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-01 13:16:43.160892: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
no_of_patients=75

In [3]:
train=sorted(glob('files/*.edf'))
train=train[:no_of_patients]
train

['files/S001R03.edf',
 'files/S002R03.edf',
 'files/S003R03.edf',
 'files/S004R03.edf',
 'files/S005R03.edf',
 'files/S006R03.edf',
 'files/S007R03.edf',
 'files/S008R03.edf',
 'files/S009R03.edf',
 'files/S010R03.edf',
 'files/S011R03.edf',
 'files/S012R03.edf',
 'files/S013R03.edf',
 'files/S014R03.edf',
 'files/S015R03.edf',
 'files/S016R03.edf',
 'files/S017R03.edf',
 'files/S018R03.edf',
 'files/S019R03.edf',
 'files/S020R03.edf',
 'files/S021R03.edf',
 'files/S022R03.edf',
 'files/S023R03.edf',
 'files/S024R03.edf',
 'files/S025R03.edf',
 'files/S026R03.edf',
 'files/S027R03.edf',
 'files/S028R03.edf',
 'files/S029R03.edf',
 'files/S030R03.edf',
 'files/S031R03.edf',
 'files/S032R03.edf',
 'files/S033R03.edf',
 'files/S034R03.edf',
 'files/S035R03.edf',
 'files/S036R03.edf',
 'files/S037R03.edf',
 'files/S038R03.edf',
 'files/S039R03.edf',
 'files/S040R03.edf',
 'files/S041R03.edf',
 'files/S042R03.edf',
 'files/S043R03.edf',
 'files/S044R03.edf',
 'files/S045R03.edf',
 'files/S0

In [4]:
train_split=0.8

In [5]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample(frac=1)
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [6]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [7]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [8]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [9]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(900000, 112425, 900000, 112425)

In [10]:
print(xtest.iloc[:,-1].values)

[-5.7e-05 -4.5e-05 -2.9e-05 ...  5.9e-05 -8.0e-06  1.4e-05]


In [11]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000048,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000054,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000065,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000075,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,-0.000069,-0.000027,-0.000012,0.000014,0.000003,-0.000085,0.000053,-0.000061,-0.000040,-0.000025,...,0.000008,-0.000014,0.000031,0.000005,-0.000013,0.000002,0.000021,0.000050,-0.000034,-0.000002
899996,-0.000083,-0.000031,-0.000012,0.000024,0.000006,-0.000078,0.000030,-0.000059,-0.000030,-0.000021,...,-0.000024,-0.000029,0.000003,-0.000020,-0.000035,-0.000036,0.000001,0.000025,-0.000039,0.000013
899997,-0.000001,-0.000066,-0.000030,0.000001,-0.000022,-0.000072,-0.000006,-0.000043,-0.000043,-0.000039,...,0.000013,0.000001,0.000023,-0.000005,-0.000025,-0.000016,0.000022,0.000055,-0.000040,0.000014
899998,-0.000065,-0.000056,-0.000013,0.000015,-0.000005,-0.000077,0.000008,-0.000069,-0.000028,-0.000022,...,-0.000033,-0.000025,-0.000007,-0.000031,-0.000048,-0.000048,-0.000012,0.000034,-0.000044,-0.000002


In [12]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [13]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [14]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/tmp/ipykernel_25398/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/tmp/ipykernel_25398/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/tmp/ipykernel_25398/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000048,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000054,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000065,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000075,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,-0.000069,-0.000027,-0.000012,0.000014,0.000003,-0.000085,0.000053,-0.000061,-0.000040,-0.000025,...,0.000008,-0.000014,0.000031,0.000005,-0.000013,0.000002,0.000021,0.000050,-0.000034,-0.000002
899996,-0.000083,-0.000031,-0.000012,0.000024,0.000006,-0.000078,0.000030,-0.000059,-0.000030,-0.000021,...,-0.000024,-0.000029,0.000003,-0.000020,-0.000035,-0.000036,0.000001,0.000025,-0.000039,0.000013
899997,-0.000001,-0.000066,-0.000030,0.000001,-0.000022,-0.000072,-0.000006,-0.000043,-0.000043,-0.000039,...,0.000013,0.000001,0.000023,-0.000005,-0.000025,-0.000016,0.000022,0.000055,-0.000040,0.000014
899998,-0.000065,-0.000056,-0.000013,0.000015,-0.000005,-0.000077,0.000008,-0.000069,-0.000028,-0.000022,...,-0.000033,-0.000025,-0.000007,-0.000031,-0.000048,-0.000048,-0.000012,0.000034,-0.000044,-0.000002


In [15]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [16]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(8, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(75, activation = 'linear', name = "L6"),
    ]
)

In [17]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=10,validation_data=(xv8,yv8)
)

Epoch 1/10


2023-10-01 13:16:49.421721: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28800000 exceeds 10% of free system memory.


28125/28125 [==============================] - 55s 2ms/step - loss: 2.8225 - val_loss: 2.8317
Epoch 2/10
28125/28125 [==============================] - 48s 2ms/step - loss: 2.3176 - val_loss: 2.6093
Epoch 3/10
28125/28125 [==============================] - 51s 2ms/step - loss: 2.0972 - val_loss: 2.6304
Epoch 4/10
28125/28125 [==============================] - 55s 2ms/step - loss: 2.0274 - val_loss: 2.6406
Epoch 5/10
28125/28125 [==============================] - 62s 2ms/step - loss: 1.9869 - val_loss: 2.6239
Epoch 6/10
28125/28125 [==============================] - 60s 2ms/step - loss: 1.9615 - val_loss: 2.6328
Epoch 7/10
28125/28125 [==============================] - 49s 2ms/step - loss: 1.9420 - val_loss: 2.6870
Epoch 8/10
28125/28125 [==============================] - 51s 2ms/step - loss: 1.9262 - val_loss: 2.6899
Epoch 9/10
28125/28125 [==============================] - 49s 2ms/step - loss: 1.9133 - val_loss: 2.7013
Epoch 10/10
28125/28125 [==============================] - 51s 2ms

In [18]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

  52/3514 [..............................] - ETA: 3s  

3514/3514 [==============================] - 4s 997us/step


2023-10-01 13:25:44.320311: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33727500 exceeds 10% of free system memory.


              precision    recall  f1-score   support

           0       0.16      0.13      0.15      1499
           1       0.19      0.10      0.13      1499
           2       0.34      0.11      0.17      1499
           3       0.66      0.48      0.56      1499
           4       0.67      0.64      0.65      1499
           5       0.40      0.37      0.38      1499
           6       0.42      0.49      0.45      1499
           7       0.48      0.46      0.47      1499
           8       0.55      0.56      0.56      1499
           9       0.54      0.25      0.34      1499
          10       0.28      0.42      0.34      1499
          11       0.18      0.26      0.22      1499
          12       0.28      0.23      0.25      1499
          13       0.16      0.11      0.13      1499
          14       0.21      0.03      0.05      1499
          15       0.41      0.35      0.38      1499
          16       0.04      0.03      0.04      1499
          17       0.32    

2023-10-01 13:25:44.760401: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33727500 exceeds 10% of free system memory.
2023-10-01 13:25:44.766960: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33727500 exceeds 10% of free system memory.


## 0-16

In [19]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [20]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/tmp/ipykernel_25398/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/tmp/ipykernel_25398/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/tmp/ipykernel_25398/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000057,-0.000013,-0.000015,-0.000012,-0.000013,-0.000008,-0.000040,-0.000054,-0.000012,-0.000014,...,-0.000048,-0.000038,-0.000042,-0.000068,-0.000076,-0.000103,-0.000051,-0.000056,-0.000124,-0.000028
1,-0.000049,-0.000011,-0.000010,-0.000012,-0.000019,-0.000024,-0.000058,-0.000051,-0.000019,-0.000023,...,-0.000055,-0.000055,-0.000063,-0.000082,-0.000087,-0.000099,-0.000059,-0.000070,-0.000149,-0.000040
2,-0.000055,-0.000017,-0.000016,-0.000019,-0.000024,-0.000029,-0.000066,-0.000061,-0.000030,-0.000036,...,-0.000054,-0.000063,-0.000072,-0.000091,-0.000092,-0.000091,-0.000067,-0.000077,-0.000153,-0.000037
3,-0.000073,-0.000042,-0.000040,-0.000037,-0.000037,-0.000040,-0.000071,-0.000078,-0.000053,-0.000053,...,-0.000065,-0.000052,-0.000066,-0.000100,-0.000105,-0.000105,-0.000067,-0.000072,-0.000148,-0.000026
4,-0.000087,-0.000053,-0.000052,-0.000051,-0.000045,-0.000043,-0.000071,-0.000087,-0.000065,-0.000064,...,-0.000075,-0.000082,-0.000090,-0.000117,-0.000119,-0.000118,-0.000075,-0.000082,-0.000161,-0.000035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,-0.000069,-0.000027,-0.000012,0.000014,0.000003,-0.000085,0.000053,-0.000061,-0.000040,-0.000025,...,0.000008,-0.000014,0.000031,0.000005,-0.000013,0.000002,0.000021,0.000050,-0.000034,-0.000002
899996,-0.000083,-0.000031,-0.000012,0.000024,0.000006,-0.000078,0.000030,-0.000059,-0.000030,-0.000021,...,-0.000024,-0.000029,0.000003,-0.000020,-0.000035,-0.000036,0.000001,0.000025,-0.000039,0.000013
899997,-0.000001,-0.000066,-0.000030,0.000001,-0.000022,-0.000072,-0.000006,-0.000043,-0.000043,-0.000039,...,0.000013,0.000001,0.000023,-0.000005,-0.000025,-0.000016,0.000022,0.000055,-0.000040,0.000014
899998,-0.000065,-0.000056,-0.000013,0.000015,-0.000005,-0.000077,0.000008,-0.000069,-0.000028,-0.000022,...,-0.000033,-0.000025,-0.000007,-0.000031,-0.000048,-0.000048,-0.000012,0.000034,-0.000044,-0.000002


In [21]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [22]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(16, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(75, activation = 'linear', name = "L6"),
    ]
)

In [23]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=10,validation_data=(xv16,yv16)
)

Epoch 1/10


2023-10-01 13:25:45.529341: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 57600000 exceeds 10% of free system memory.


28125/28125 [==============================] - 51s 2ms/step - loss: 2.0967 - val_loss: 2.6360
Epoch 2/10
28125/28125 [==============================] - 50s 2ms/step - loss: 1.4131 - val_loss: 2.4992
Epoch 3/10
28125/28125 [==============================] - 49s 2ms/step - loss: 1.2081 - val_loss: 2.4241
Epoch 4/10
28125/28125 [==============================] - 51s 2ms/step - loss: 1.1178 - val_loss: 2.4097
Epoch 5/10
28125/28125 [==============================] - 49s 2ms/step - loss: 1.0646 - val_loss: 2.5147
Epoch 6/10
28125/28125 [==============================] - 50s 2ms/step - loss: 1.0283 - val_loss: 2.6538
Epoch 7/10
28125/28125 [==============================] - 50s 2ms/step - loss: 1.0006 - val_loss: 2.5147
Epoch 8/10
28125/28125 [==============================] - 50s 2ms/step - loss: 0.9787 - val_loss: 2.6059
Epoch 9/10
28125/28125 [==============================] - 51s 2ms/step - loss: 0.9583 - val_loss: 2.7531
Epoch 10/10
28125/28125 [==============================] - 50s 2ms

In [24]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

  48/3514 [..............................] - ETA: 3s  

3514/3514 [==============================] - 4s 1ms/step
              precision    recall  f1-score   support

           0       0.28      0.52      0.37      1499
           1       0.33      0.39      0.36      1499
           2       0.29      0.20      0.24      1499
           3       0.68      0.34      0.45      1499
           4       0.83      0.80      0.81      1499
           5       0.62      0.50      0.55      1499
           6       0.64      0.78      0.70      1499
           7       0.77      0.67      0.72      1499
           8       0.48      0.68      0.56      1499
           9       0.46      0.52      0.49      1499
          10       0.50      0.39      0.44      1499
          11       0.57      0.39      0.46      1499
          12       0.37      0.40      0.39      1499
          13       0.30      0.28      0.29      1499
          14       0.30      0.15      0.20      1499
          15       0.53      0.12      0.19      1499
          16       0.29 

## 0-32

In [25]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [26]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/tmp/ipykernel_25398/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/tmp/ipykernel_25398/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/tmp/ipykernel_25398/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [27]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

In [28]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(75, activation = 'linear', name = "L6"),
    ]
)

In [29]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=10,validation_data=(xv32,yv32)
)

Epoch 1/10
28125/28125 [==============================] - 39s 1ms/step - loss: 1.3949 - val_loss: 2.0504
Epoch 2/10
28125/28125 [==============================] - 45s 2ms/step - loss: 0.6215 - val_loss: 2.0341
Epoch 3/10
28125/28125 [==============================] - 51s 2ms/step - loss: 0.4638 - val_loss: 1.9311
Epoch 4/10
28125/28125 [==============================] - 51s 2ms/step - loss: 0.3971 - val_loss: 2.0182
Epoch 5/10
28125/28125 [==============================] - 51s 2ms/step - loss: 0.3580 - val_loss: 1.9310
Epoch 6/10
28125/28125 [==============================] - 52s 2ms/step - loss: 0.3303 - val_loss: 1.8785
Epoch 7/10
28125/28125 [==============================] - 50s 2ms/step - loss: 0.3107 - val_loss: 1.9678
Epoch 8/10
28125/28125 [==============================] - 51s 2ms/step - loss: 0.2952 - val_loss: 1.8815
Epoch 9/10
28125/28125 [==============================] - 51s 2ms/step - loss: 0.2813 - val_loss: 1.8971
Epoch 10/10
28125/28125 [==============================

In [30]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

  48/3514 [..............................] - ETA: 3s  

3514/3514 [==============================] - 4s 1ms/step
              precision    recall  f1-score   support

           0       0.74      0.72      0.73      1499
           1       0.58      0.72      0.64      1499
           2       0.70      0.56      0.62      1499
           3       0.76      0.44      0.56      1499
           4       0.93      0.91      0.92      1499
           5       0.99      0.76      0.86      1499
           6       0.98      0.81      0.89      1499
           7       0.86      0.42      0.56      1499
           8       0.32      0.74      0.45      1499
           9       0.66      0.77      0.71      1499
          10       0.63      0.90      0.74      1499
          11       0.87      0.84      0.86      1499
          12       0.69      0.67      0.68      1499
          13       0.83      0.77      0.80      1499
          14       0.56      0.62      0.59      1499
          15       0.72      0.29      0.41      1499
          16       0.17 

## 0-64

In [31]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [32]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [33]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(75, activation = 'linear', name = "L6"),
    ]
)

In [34]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

Epoch 1/10
28125/28125 [==============================] - 53s 2ms/step - loss: 1.0463 - val_loss: 2.0202
Epoch 2/10
28125/28125 [==============================] - 49s 2ms/step - loss: 0.4225 - val_loss: 2.2758
Epoch 3/10
28125/28125 [==============================] - 40s 1ms/step - loss: 0.3050 - val_loss: 2.0507
Epoch 4/10
28125/28125 [==============================] - 40s 1ms/step - loss: 0.2393 - val_loss: 1.9247
Epoch 5/10
28125/28125 [==============================] - 40s 1ms/step - loss: 0.1998 - val_loss: 1.7235
Epoch 6/10
28125/28125 [==============================] - 40s 1ms/step - loss: 0.1712 - val_loss: 1.8004
Epoch 7/10
28125/28125 [==============================] - 40s 1ms/step - loss: 0.1519 - val_loss: 1.6581
Epoch 8/10
28125/28125 [==============================] - 40s 1ms/step - loss: 0.1373 - val_loss: 1.7251
Epoch 9/10
28125/28125 [==============================] - 40s 1ms/step - loss: 0.1249 - val_loss: 1.8172
Epoch 10/10
28125/28125 [==============================

In [35]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

  60/3514 [..............................] - ETA: 2s  

3514/3514 [==============================] - 3s 883us/step
              precision    recall  f1-score   support

           0       0.84      0.81      0.82      1499
           1       0.71      0.96      0.82      1499
           2       0.62      0.68      0.65      1499
           3       0.88      0.88      0.88      1499
           4       0.91      0.99      0.95      1499
           5       0.99      0.88      0.93      1499
           6       0.97      0.98      0.98      1499
           7       0.95      0.86      0.91      1499
           8       0.43      0.89      0.58      1499
           9       0.60      0.77      0.68      1499
          10       0.85      0.91      0.88      1499
          11       0.89      0.89      0.89      1499
          12       0.77      0.86      0.81      1499
          13       0.89      0.81      0.85      1499
          14       0.68      0.87      0.76      1499
          15       0.91      0.22      0.36      1499
          16       0.6